# 크로스 밸리데이션
    * 지금까지 교제에서 해왔었던 방식은 1분할 방식이였음
        * 훈련전용 데이터와 테스트 전용 데이터로 분할한 뒤 서로 크로스로 학습의 타당성을 검증하는 방법
    * K 분할 교차 검증 ( K-fold cross vaildation )
        * K는 특정 숫자인데 보통 5~10을 사용 
        * 훈련 데이터를 K개로 나누고 그중에 한묶음을 검증데이터로 사용하고 나머지 묶음을 훈련 데이터로 사용하여 학습을 시킴
        * K번의 정답률을 구할 수 있으므로 더욱 신뢰성있는 값을 도출할 수있는 장점이 있음.
            * 운 나쁘게 분류하기 어려운 샘플들이 모두 테스트 세트에 들어가는 최악의 케이스 방지하도록 함.
        * 데이터셋에 있는 모든 샘플이 테스트될 기회를 갖도록 하는 것.
![Alt text](https://tensorflowkorea.files.wordpress.com/2016/10/07_cross_validation_diagram.png)


크로스 밸리데이션은 훈련과 테스트를 위해 모든 데이터를 사용
이 방식 이면에 있는 아이디어는 테스트 데이터를 위해 데이터셋에서 비교적 큰 부분을 따로 떼어 놓지 않고 
많은 훈련 데이터를 사용하여 비관적 편향을 감소시키려는 목적이 있는것 같음.

In [19]:
from sklearn import svm, metrics
import random, re

# 붓꽃의 CSV 파일 읽어 들이기 
lines = open('iris.csv', 'r', encoding='utf-8').read().split("\n")

# 문자열 데이터 숫자변환 람다함수 
f_tonum = lambda n : float(n) if re.match(r'^[0-9\.]+$', n) else n
# 쉽표로 데이터 뽑아주는 람다함수 ( 공백제거 후 ',' 스플릿)
f_cols  = lambda li: list(map(f_tonum, li.strip().split(',')))
csv = list(map(f_cols, lines))

# 헤더 제거하기 (epalLength,SepalWidth,PetalLength,PetalWidth,Name)
del csv[0] 

print(type(csv))

<class 'list'>


In [32]:
# 데이터를 K개로 분할하기 

K = 5 
csvk = [ [] for i in range(K) ]
print(len(csvk))
print(type(csvk[0])) 

print(len(csv))
for i in range(len(csv)):
    csvk[i % K].append(csv[i])



5
<class 'list'>
150


In [38]:
# 리스트를 훈련 전용 데이터와 테스트 전용 데이터로 분할하는 함수
def split_data_label(rows):
    data = []; label = []
    for row in rows:
        data.append(row[0:4])
        label.append(row[4])
    return (data, label)

# 정답률 구하는 함수 
def calc_score(test, train):
    test_f, test_l = split_data_label(test)
    train_f, train_l = split_data_label(train)
    # 학습시키고 정답률 구하기
    clf = svm.SVC()
    clf.fit(train_f, train_l)
    pre = clf.predict(test_f)
    return metrics.accuracy_score(test_l, pre)

# K개로 분할해서 정답률 구하기
score_list = []
for testc in csvk:
    # testc 이외의 데이터를 훈련 전용 데이터로 사용하기
    trainc = []
    for i in csvk:
        if i != testc: trainc += i
    sc = calc_score(testc, trainc)
    score_list.append(sc)
    
print("각각의 정답률 =", score_list)
print("평균 정답률 =", sum(score_list) / len(score_list))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
각각의 정답률 = [0.9666666666666667, 0.9333333333333333, 0.9666666666666667, 0.9666666666666667, 1.0]
평균 정답률 = 0.9666666666666668


# scikit-learn 의 크로스 밸리데이션

* API 
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

    * cross_val_score() 함수 사용 및 이해는 소스를 통해서 확인


In [4]:
import pandas as pd
from sklearn import svm, metrics, model_selection
import random, re

# 붓꽃의 CSV 데이터 읽어 들이기 
csv = pd.read_csv('iris.csv')

# 리스트를 훈련 전용 데이터와 테스트 전용 데이터로 분할하기 
data = csv[["SepalLength","SepalWidth","PetalLength","PetalWidth"]]
# print(data)
label = csv["Name"]
# print(label)

# 크로스 밸리데이션하기
clf = svm.SVC()
scores = model_selection.cross_val_score(clf, data, label, cv=5)
print("각각의 정답률 =", scores)
print("평균 정답률 =", scores.mean())

각각의 정답률 = [0.96666667 1.         0.96666667 0.96666667 1.        ]
평균 정답률 = 0.9800000000000001


In [5]:
from sklearn.datasets import load_iris
from sklearn import svm, metrics, model_selection

# scikit-learn 의 붓꽃 데이터셋을 제공해줌 ( 다른 데이터셋도 많다. )
# http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html
iris = load_iris()

clf = svm.SVC()
scores = model_selection.cross_val_score(clf, iris.data, iris.target, cv=3)

print("각각의 정답률 =", scores)
print("평균 정답률 =", scores.mean())


각각의 정답률 = [0.96666667 1.         0.96666667 0.96666667 1.        ]
평균 정답률 = 0.9800000000000001


# 교차 검증의 상세 옵션
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

* KFold
 * 데이터 섞어주는 기능 추가 
* ShuffleSplit
 * 반복횟수, 훈련세트나 테스트 세트의 크기를 독립적으로 조절할 때 유용 

In [7]:
# 순서대로 K개의 폴드로 나누는것이 항상 좋은 결과를 주지 않음.
# 결과를 보더라도 순차적으로 K=3으로 했을 경우 첫번째 반복에서 0%으로 나올 활률이 있음 
# A와 B의 구성이 90대 10으로 구성이 나뉜다면 A의 샘플만 가진 폴드가 생길수 있기때문에 분류기의 전체성능이 왜곡하게 됨 
print("Iris 레이블:\n{}".format(iris.target))

Iris 레이블:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [18]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=3)
print("교차 검증 점수:\n{}".format(
      cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

#  데이터를 섞어서 샘플의 순서를 뒤죽박죽으로 만들어야 함.
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("교차 검증 점수:\n{}".format(
    cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

교차 검증 점수:
[0. 0. 0.]
교차 검증 점수:
[0.9  0.96 0.96]


In [21]:
from sklearn.model_selection import ShuffleSplit

# 데이터셋의 50%를 훈련세트, 50%를 테스트 세트로 10번 반복 분할 처리 
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)
print("교차 검증 점수:\n{}".format(scores))

교차 검증 점수:
[0.86666667 0.97333333 0.92       0.90666667 0.89333333 0.92
 0.93333333 0.94666667 0.94666667 0.97333333]


# SVM ( Support Vector Machine )
http://bskyvision.com/163

#### 매개변수
 * C : 데이터 샘플들이 다른 클래스에 놓이는 것을 허용하는 정도를 결정
 * gamma : 결정 경계의 곡률을 결정



# 그리드 서치 ( GridSearchCV )
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

![Alt text](http://cfile23.uf.tistory.com/image/27795E4F5915750E064D09)

### 동작
* 클래스 객체의 fit매서드를 호출하면 grid search를 사용하여 자동으로 복수개의 내부모형을 생성하고 이를 모두 실행시켜서 최적의 파라미터를 찾아주고 다음 속성의 저장함, 따라서 시간이 꽤 오래걸리기 때문에 n_jobs을 이용하여 처리하면 된다고 함 
* n_jobs(디폴트 값은 1) : 병렬 계산할 프로세스 수 지정 ( 
    * 이 값을 증가시키면 내부적으로 멀티 프로세스를 사용하여 그리드서치를 수행 
    * 만약 CPU 코어의 수가 충분하다면 n_jobs를 늘릴 수록 속도가 증가
    * 값을 -1로 지정할 경우 자동으로 코어의 수에 맞게 프로세스 수를 정해줌 

### 속성
* grid_scores_
    * param_grid 의 모든 파리미터 조합에 대한 성능 결과. 각각의 원소는 다음 요소로 이루어진 튜플이다.
    * parameters: 사용된 파라미터
    * mean_validation_score: 교차 검증(cross-validation) 결과의 평균값
    * cv_validation_scores: 모든 교차 검증(cross-validation) 결과
* best_score_
    * 최고 점수
* best_params_
    * 최고 점수를 낸 파라미터
* best_estimator_
    * 최고 점수를 낸 파라미터를 가진 모형
    
### 정리
* SVM는 딥러닝이 나온 후에도 여전히 환영받는 머신러닝 알고리즘 ( 가볍기 때문 )
* SVM 알고리즘 중에서 가장 성능이 괜찮고좋은 성능을 얻으려면 매개변수인 C와 gamma를 잘 조정해 줘야하는것이 키포인트 
* 각 알고리즘에는 여러 매개변수를 지정하는데 적잘한 매개변수를 지정하면 정답률이 올라가게 됨. 
* 그리드 서치는 더 좋은 매개변수를 자동으로 찾는 방법임
    * 각 매개변수를 적당한 범위 내부에서 변경하면서 가장 성능 좋을 때의 값을 찾는 방법 
    * 매개변수 튜닝 작업임
* 기존의 손글씨 인식 프로그램을 비교하여 매개변수를 어떻게 튜닝하면 좋은 결과가 나오는지가 이 파트의 핵심임


#### 참고링크
* https://datascienceschool.net/view-notebook/ff4b5d491cc34f94aea04baca86fbef8/
* http://cyan91.tistory.com/18

# 간단한 그리드 서치
* SVC 에 구현된 SVM 사용 
 * http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
 * default : RBF (Radial Basis Function) 커널 SVM 

In [3]:
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn import svm, metrics, model_selection
from sklearn.model_selection import train_test_split

iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
print("훈련 세트의 크기: {}   테스트 세트의 크기: {}".format(X_train.shape[0], X_test.shape[0]))

best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # 매개변수의 각 조합에 대해 SVC를 훈련시킵니다
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        # 테스트 세트로 SVC를 평가합니다
        score = svm.score(X_test, y_test)
        
        # 점수가 더 높으면 매개변수와 함께 기록합니다
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}
            
print("최고 점수: {:.2f}".format(best_score))
print("최적 파라미터: {}".format(best_parameters))


훈련 세트의 크기: 112   테스트 세트의 크기: 38
최고 점수: 0.97
최적 파라미터: {'C': 100, 'gamma': 0.001}


## 이전에 사용했던 손글씨 인식 프로그램
* 그리드 서치를 했을때와 안했을때의 정답률이 어떻게 개선되는지 확인 

In [35]:
from sklearn import model_selection, svm, metrics

def load_csv(fname):
    labels = []
    images = []
    with open(fname, "r") as f:
        for line in f:
            cols = line.split(",")
            if len(cols) < 2: continue
            labels.append(int(cols.pop(0)))
            vals = list(map(lambda n: int(n) / 256, cols))
            images.append(vals)
    return {"labels":labels, "images":images}

data = load_csv("./mnist/train.csv")
test = load_csv("./mnist/t10k.csv")

# 학습하기 
clf = svm.SVC()
clf.fit(data["images"], data["labels"])
# 예측하기 
predict = clf.predict(test["images"])
# 결과 확인하기 
ac_score = metrics.accuracy_score(test["labels"], predict)
print(">>> 정답률 =", ac_score)

>>> 정답률 = 0.7884231536926147


## 그리드 서치를 활용한 정답률 개선 방법

In [40]:
from sklearn import model_selection, svm, metrics
from sklearn.grid_search import GridSearchCV

# 그리드 서치 매개변수 설정 ( 매개변수 후보들 ) 
params = [
    {"C": [1,10,100,1000], "kernel":["linear"]},
    {"C": [1,10,100,1000], "kernel":["rbf"], "gamma":[0.001, 0.0001]}
]

# 그리드 서치 수행 
clf = GridSearchCV( svm.SVC(), params, n_jobs=-1 )
clf.fit(data["images"], data["labels"])
print("> grid_scores =", clf.grid_scores_)
print("> best_score =", clf.best_score_)
print("> best_params =", clf.best_params_)
print("> best_estimator =", clf.best_estimator_)

# 테스트 데이터 확인하기 
pre = clf.predict(test["images"])
ac_score = metrics.accuracy_score(pre, test["labels"])
print(">>> 정답률 =", ac_score)

> grid_scores = [mean: 0.86813, std: 0.00225, params: {'C': 1, 'kernel': 'linear'}, mean: 0.86813, std: 0.00225, params: {'C': 10, 'kernel': 'linear'}, mean: 0.86813, std: 0.00225, params: {'C': 100, 'kernel': 'linear'}, mean: 0.86813, std: 0.00225, params: {'C': 1000, 'kernel': 'linear'}, mean: 0.79421, std: 0.00831, params: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}, mean: 0.14885, std: 0.04624, params: {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}, mean: 0.87013, std: 0.00918, params: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}, mean: 0.79820, std: 0.00966, params: {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}, mean: 0.87612, std: 0.01185, params: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}, mean: 0.87013, std: 0.00788, params: {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}, mean: 0.87612, std: 0.01185, params: {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}, mean: 0.87013, std: 0.00505, params: {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}]
> best_score = 0.8761238761238761
> best_params = {'C